# 볼린저 밴드 함수화
1. 첫 번째 함수를 생성(매개변수 데이터프레임 하나)
    1. 결측치,이상치 제거한다.
    2. 컬럼을 종가만 두고 모두 삭제
    3. 이동평균선, 상단밴드, 하단밴드 생성
    
2. 두 번째 함수를 생성(매개변수 데이터프레임 하나)
    1. 거래 내역이라는 파생 변수 생성 값은  ""
    2. 볼린저 밴드의 거래 내역을 추가
    3. 거래 내역을 출력(print)

3. 세 번째 함수를 생성(매개변수 데이터프레임 하나)
    1. 수익율 파생변수 생성 값은 "1"
    2. 판매를 한 날의 수익율 변경
    3. 누적 수익율을 계산하여 새로운 파생변수를 생성
    4. 최종 누적 수익율을 출력(print)

In [79]:
import pandas as pd
import numpy as np
from datetime import datetime

In [69]:
# 함수 1
def bollinger1(df,col):
    df=df.loc[~df.isin((np.nan,np.inf,-np.inf)).any(axis=1)]
    price_df=df[[col]]  # price_df라는 변수 1-2
    price_df['center']=price_df[col].rolling(20).mean() # 이동평균성 파생변수 생성
    price_df['ub']=price_df['center']+(2*price_df[col].rolling(20).std())   # 상단 밴드 파생변수
    price_df['lb']=price_df['center']-(2*price_df[col].rolling(20).std())   # 하단 밴드 파생변수
    

    return price_df





In [70]:
df = pd.read_csv('../csv/GM.csv', index_col='Date')
df=bollinger1(df,"Close")
df

C:\Users\dgh06\AppData\Local\Temp\ipykernel_1436\661784885.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price_df['center']=price_df[col].rolling(20).mean() # 이동평균성 파생변수 생성
C:\Users\dgh06\AppData\Local\Temp\ipykernel_1436\661784885.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price_df['ub']=price_df['center']+(2*price_df[col].rolling(20).std())   # 상단 밴드 파생변수
C:\Users\dgh06\AppData\Local\Temp\ipykernel_1436\661784885.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

,Close,center,ub,lb
Date,,,,
2010-11-18,34.189999,NaN,NaN,NaN
2010-11-19,34.259998,NaN,NaN,NaN
2010-11-22,34.080002,NaN,NaN,NaN
2010-11-23,33.250000,NaN,NaN,NaN
2010-11-24,33.480000,NaN,NaN,NaN
...,...,...,...,...
2019-06-18,36.700001,35.45400,37.253548,33.654451
2019-06-19,36.779999,35.43650,37.173130,33.699870
2019-06-20,36.959999,35.50700,37.372713,33.641286


In [71]:
# 함수 2
def bollinger2(price_df,col):
    price_df['trade']=""    # 거래 내역이라는 파생변수

    for i in price_df.index:
    # 상단 밴드보다 종가가 높은 경우
        if price_df.loc[i,col]> price_df.loc[i,"ub"]:
            # 현재 구매 상태이면
            if price_df.shift(1).loc[i,'trade']=='buy':
                #매도
                price_df.loc[i,'trade']=''
            else:
                price_df.loc[i,'trade']=''
            
            # 하단 밴드보다 종가가 낮은 경우
        elif price_df.loc[i,col]< price_df.loc[i,'lb']:
            # 현재 구매 상태이면
            if price_df.shift(1).loc[i,'trade']=="buy":
                # 구매 상태를 유지
                price_df.loc[i,'trade']= 'buy'
            else:
                price_df.loc[i,'trade']='buy'
        else:
            # 현재 구매 상태이면
            if price_df.shift(1).loc[i,'trade']=='buy':
                #구매 상태를 유지
                price_df.loc[i,'trade']='buy'
            else:
                price_df.loc[i,'trade']=''

    return price_df



In [72]:
df=bollinger2(df,"Close")
df

,Close,center,ub,lb,trade
Date,,,,,
2010-11-18,34.189999,NaN,NaN,NaN,
2010-11-19,34.259998,NaN,NaN,NaN,
2010-11-22,34.080002,NaN,NaN,NaN,
2010-11-23,33.250000,NaN,NaN,NaN,
2010-11-24,33.480000,NaN,NaN,NaN,
...,...,...,...,...,...
2019-06-18,36.700001,35.45400,37.253548,33.654451,buy
2019-06-19,36.779999,35.43650,37.173130,33.699870,buy
2019-06-20,36.959999,35.50700,37.372713,33.641286,buy


In [62]:
# 함수 3
def bollinger3(N_df,col):# 파생변수 N_df
    
    rtn=1.0
    N_df["return"]=1       # "return"이라는 파생변수 추가
    buy=0.0
    sell=0.0
    for i in N_df.index:
        # 구매가를 출력
        if (N_df.shift(1).loc[i,'trade']=="") and (N_df.loc[i,'trade']=="buy"):
            buy =N_df.loc[i,col]
            # print('진입일 :',i, '구매가격 :',buy)
        # 판매가를 출력
        elif (N_df.shift(1).loc[i,'trade']=="buy") and (N_df.loc[i,'trade']==""):
            sell=N_df.loc[i,col]
            rtn =(sell-buy)/buy +1
            N_df.loc[i,'return'] =rtn
            # print('판매일 :',i, "판매가격 :",sell, '수익율 :', rtn)

        # 구매가, 판매가 초기화
        if N_df.loc[i,'trade']=='':
            buy=0.0
            sell=0.0

    # 누적 수익율

    acc_rtn =1.0     # 누적 수익률

    for i in N_df.index:
        rtn=N_df.loc[i,'return']
        acc_rtn *=rtn
        N_df.loc[i,"acc_rtn"]=acc_rtn


    print('누적 수익율:', acc_rtn)

In [63]:
bollinger3(df,"Close")

누적 수익율: 1.1480914103747033


In [65]:
df = pd.read_csv('../csv/GDX.csv', index_col='Date')
df=bollinger1(df,'Close')
df=bollinger2(df,"Close")
bollinger3(df,"Close")



C:\Users\dgh06\AppData\Local\Temp\ipykernel_1436\3561552184.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price_df['center']=price_df[col].rolling(20).mean() # 이동평균성 파생변수 생성
C:\Users\dgh06\AppData\Local\Temp\ipykernel_1436\3561552184.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price_df['ub']=price_df['center']+(2*price_df[col].rolling(20).std())   # 상단 밴드 파생변수
C:\Users\dgh06\AppData\Local\Temp\ipykernel_1436\3561552184.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

누적 수익율: 2.687593657120985


In [76]:
df = pd.read_csv('../csv/GM.csv', index_col='Date')
df=bollinger1(df,"Close")
df

C:\Users\dgh06\AppData\Local\Temp\ipykernel_1436\1302264285.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price_df['center']=price_df[col].rolling(20).mean() # 이동평균성 파생변수 생성
C:\Users\dgh06\AppData\Local\Temp\ipykernel_1436\1302264285.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price_df['ub']=price_df['center']+(2*price_df[col].rolling(20).std())   # 상단 밴드 파생변수
C:\Users\dgh06\AppData\Local\Temp\ipykernel_1436\1302264285.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

,Close,center,ub,lb
Date,,,,
2010-11-18,34.189999,NaN,NaN,NaN
2010-11-19,34.259998,NaN,NaN,NaN
2010-11-22,34.080002,NaN,NaN,NaN
2010-11-23,33.250000,NaN,NaN,NaN
2010-11-24,33.480000,NaN,NaN,NaN
...,...,...,...,...
2019-06-18,36.700001,35.45400,37.253548,33.654451
2019-06-19,36.779999,35.43650,37.173130,33.699870
2019-06-20,36.959999,35.50700,37.372713,33.641286


In [74]:
# 함수 1
def bollinger1(df,col):
    df=df.loc[~df.isin((np.nan,np.inf,-np.inf)).any(axis=1)]
    # df=df.loc[~df.isin((np.nan,np.inf,-np.inf)).any(axis=1),[]]
    price_df=df[[col]]  # price_df라는 변수 1-2
    price_df['center']=price_df[col].rolling(20).mean() # 이동평균성 파생변수 생성
    price_df['ub']=price_df['center']+(2*price_df[col].rolling(20).std())   # 상단 밴드 파생변수
    price_df['lb']=price_df['center']-(2*price_df[col].rolling(20).std())   # 하단 밴드 파생변수
    

    return price_df


# 함수 2
def bollinger2(price_df):
    price_df['trade']=""    # 거래 내역이라는 파생변수

    for i in price_df.index:
    # 상단 밴드보다 종가가 높은 경우
        if price_df.iloc[i,0]> price_df.loc[i,"ub"]:
            # 현재 구매 상태이면
            if price_df.shift(1).loc[i,'trade']=='buy':
                #매도
                price_df.loc[i,'trade']=''
            else:
                price_df.loc[i,'trade']=''
            
            # 하단 밴드보다 종가가 낮은 경우
        elif price_df.loc[i,0]< price_df.loc[i,'lb']:
            # 현재 구매 상태이면
            if price_df.shift(1).loc[i,'trade']=="buy":
                # 구매 상태를 유지
                price_df.loc[i,'trade']= 'buy'
            else:
                price_df.loc[i,'trade']='buy'
        else:
            # 현재 구매 상태이면
            if price_df.shift(1).loc[i,'trade']=='buy':
                #구매 상태를 유지
                price_df.loc[i,'trade']='buy'
            else:
                price_df.loc[i,'trade']=''

    return price_df

# 함수 3
def bollinger3(N_df,col):# 파생변수 N_df
    
    rtn=1.0
    N_df["return"]=1       # "return"이라는 파생변수 추가
    buy=0.0
    sell=0.0
    for i in N_df.index:
        # 구매가를 출력
        if (N_df.shift(1).loc[i,'trade']=="") and (N_df.loc[i,'trade']=="buy"):
            buy =N_df.loc[i,col]
            # print('진입일 :',i, '구매가격 :',buy)
        # 판매가를 출력
        elif (N_df.shift(1).loc[i,'trade']=="buy") and (N_df.loc[i,'trade']==""):
            sell=N_df.loc[i,col]
            rtn =(sell-buy)/buy +1
            N_df.loc[i,'return'] =rtn
            # print('판매일 :',i, "판매가격 :",sell, '수익율 :', rtn)

        # 구매가, 판매가 초기화
        if N_df.loc[i,'trade']=='':
            buy=0.0
            sell=0.0

    # 누적 수익율

    acc_rtn =1.0     # 누적 수익률

    for i in N_df.index:
        rtn=N_df.loc[i,'return']
        acc_rtn *=rtn
        N_df.loc[i,"acc_rtn"]=acc_rtn


    print('누적 수익율:', acc_rtn)


In [75]:
df = pd.read_csv('../csv/GDX.csv', index_col='Date')
df=bollinger1(df,'Close')
df=bollinger2(df,"Close")
bollinger3(df,"Close")


C:\Users\dgh06\AppData\Local\Temp\ipykernel_1436\1302264285.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price_df['center']=price_df[col].rolling(20).mean() # 이동평균성 파생변수 생성
C:\Users\dgh06\AppData\Local\Temp\ipykernel_1436\1302264285.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price_df['ub']=price_df['center']+(2*price_df[col].rolling(20).std())   # 상단 밴드 파생변수
C:\Users\dgh06\AppData\Local\Temp\ipykernel_1436\1302264285.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

누적 수익율: 2.687593657120985
